# **Assignment 2**

This assignment focuses on Airbnbs in Berlin, Germany. The goal is to help a company operating small and mid-size apartments hosting 2-6 guests. The company is set to price their new apartments not on the market. This assignment will build a prediction model for prices, and discuss the modeling decisions and compare the results to those of the case study. 

Task
• You may use other variables we used in class.

• You may do different feature engineering depending on the selected environment.

• You may make other sample design decisions!

• In each case, document your steps!

• Have at least 3 different models and compare performance

• Argue for your choice of models
- One model must be theoretically profound linear regression via OLS.
- One model must be Random Forest or any boosting algorithm

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
from patsy import dmatrices
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [43]:
# DATA IMPORT - FROM GITHUB
data = pd.read_csv('https://github.com/Iandrewburg/Assignment_1/raw/main/Assignment_2/berlin_airbnb.csv')

In [44]:
data.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3176.0,https://www.airbnb.com/rooms/3176,2.023090e+13,16/09/2023,city scrape,Rental unit in Berlin · ★4.63 · 1 bedroom · 2 ...,This beautiful first floor apartment is situa...,The neighbourhood is famous for its variety of...,https://a0.muscache.com/pictures/243355/84afcf...,3718,...,4.69,4.92,4.62,First name and Last name: Nicolas Krotz <br/> ...,f,1,1,0,0,0.85
1,9991.0,https://www.airbnb.com/rooms/9991,2.023090e+13,16/09/2023,city scrape,Rental unit in Berlin · ★5.0 · 4 bedrooms · 7 ...,4 bedroom with very large windows and outstand...,Prenzlauer Berg is an amazing neighbourhood wh...,https://a0.muscache.com/pictures/42799131/59c8...,33852,...,5.00,4.86,4.86,03/Z/RA/003410-18,f,1,1,0,0,0.08
2,14325.0,https://www.airbnb.com/rooms/14325,2.023090e+13,16/09/2023,city scrape,Rental unit in Berlin · ★4.67 · Studio · 1 bed...,The apartment is located on the upper second f...,NaN,https://a0.muscache.com/pictures/508703/24988a...,55531,...,4.84,4.58,4.42,NaN,f,4,4,0,0,0.16
3,16644.0,https://www.airbnb.com/rooms/16644,2.023090e+13,16/09/2023,city scrape,Condo in Berlin · ★4.72 · 1 bedroom · 1 bed · ...,Light and sunny 2-Room-turn of the century-fla...,Our Part of Kreuzberg is just the best. Good v...,https://a0.muscache.com/pictures/73759174/e2ef...,64696,...,4.86,4.67,4.71,NaN,f,2,2,0,0,0.30
4,17904.0,https://www.airbnb.com/rooms/17904,2.023090e+13,16/09/2023,city scrape,Rental unit in Berlin · ★4.77 · Studio · 1 bed...,"- apt is available starting April 1, 2023<br /...",NaN,https://a0.muscache.com/pictures/d9a6f8be-54b9...,68997,...,4.92,4.87,4.65,NaN,f,1,1,0,0,1.81


In [45]:
data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13134 entries, 0 to 13133
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            13134 non-null  float64
 1   listing_url                                   13134 non-null  object 
 2   scrape_id                                     13134 non-null  float64
 3   last_scraped                                  13134 non-null  object 
 4   source                                        13134 non-null  object 
 5   name                                          13134 non-null  object 
 6   description                                   12997 non-null  object 
 7   neighborhood_overview                         6771 non-null   object 
 8   picture_url                                   13134 non-null  object 
 9   host_id                                       13134 non-null 

In [47]:
data.room_type.value_counts()

room_type
Entire home/apt    8431
Private room       4309
Shared room         262
Hotel room          132
Name: count, dtype: int64

In [48]:
data.property_type.value_counts()

property_type
Entire rental unit             6545
Private room in rental unit    3261
Entire condo                    756
Entire serviced apartment       407
Room in hotel                   297
                               ... 
Shared room in condo              1
Private room in boat              1
Treehouse                         1
Island                            1
Dome                              1
Name: count, Length: 67, dtype: int64

In [49]:
data.amenities

0        ["Kitchen", "Crib", "Essentials", "Patio or ba...
1        ["Refrigerator", "TV with standard cable", "Ba...
2        ["Wifi", "TV", "Refrigerator", "Heating", "Cof...
3        ["Kitchen", "Essentials", "Wifi", "TV", "High ...
4        ["Long term stays allowed", "Kitchen", "Essent...
                               ...                        
13129    ["Wifi", "TV", "Smoke alarm", "Washer", "Kitch...
13130    ["Wifi", "TV", "Dedicated workspace", "Smoke a...
13131    ["Wifi", "TV", "Smoke alarm", "Washer", "Kitch...
13132    ["Fire extinguisher", "Wifi", "TV", "Paid park...
13133    ["Long term stays allowed", "Room-darkening sh...
Name: amenities, Length: 13134, dtype: object

In [34]:
import pandas as pd
import ast

# Assuming 'data' is your DataFrame and 'amenities' is the column with strings that look like lists

# Convert the string representations of lists into actual lists
data['amenities'] = data['amenities'].apply(lambda x: ast.literal_eval(x))

# Flatten the list to count each amenity's occurrence
all_amenities = [amenity for sublist in data['amenities'] for amenity in sublist]

# Find the top 15 most frequent amenities
amenities_series = pd.Series(all_amenities)
top_15_amenities = amenities_series.value_counts().head(15).index.tolist()

# Create binary variables for these top 15 amenities
for amenity in top_15_amenities:
    # Correctly format column names by replacing spaces with underscores
    # Ensure any potential quotes are handled correctly
    column_name = "d_" + amenity.replace(' ', '_').replace('"', '')
    data[column_name] = data['amenities'].apply(lambda x: 1 if amenity in x else 0)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13134 entries, 0 to 13133
Data columns (total 90 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            13134 non-null  float64
 1   listing_url                                   13134 non-null  object 
 2   scrape_id                                     13134 non-null  float64
 3   last_scraped                                  13134 non-null  object 
 4   source                                        13134 non-null  object 
 5   name                                          13134 non-null  object 
 6   description                                   12997 non-null  object 
 7   neighborhood_overview                         6771 non-null   object 
 8   picture_url                                   13134 non-null  object 
 9   host_id                                       13134 non-null 

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13134 entries, 0 to 13133
Columns: 106 entries, id to d_Shampoo
dtypes: float64(20), int64(50), object(36)
memory usage: 10.6+ MB


In [42]:
def airbnb_cleaner(data): 
    
    # Convert the string representations of lists into actual lists
    data['amenities'] = data['amenities'].apply(lambda x: ast.literal_eval(x))

    # Flatten the list to count each amenity's occurrence
    all_amenities = [amenity for sublist in data['amenities'] for amenity in sublist]

    # Find the top 15 most frequent amenities
    amenities_series = pd.Series(all_amenities)
    top_15_amenities = amenities_series.value_counts().head(15).index.tolist()

    # Create binary variables for these top 15 amenities
    for amenity in top_15_amenities:
        # Correctly format column names by replacing spaces with underscores
        # Ensure any potential quotes are handled correctly
        column_name = "d_" + amenity.replace(' ', '_').replace('"', '')
        data[column_name] = data['amenities'].apply(lambda x: 1 if amenity in x else 0)
    
    # Filter the data
    data = data[(data.accommodates >= 2) & (data.accommodates <= 6)]
    
    # Define the top amenities columns to keep
    top_amenities_columns = [
        'd_Kitchen', 'd_Wifi', 'd_Essentials', 'd_Hair_dryer', 'd_Heating',
        'd_Hangers', 'd_Smoke_alarm', 'd_Hot_water', 'd_Dishes_and_silverware',
        'd_Washer', 'd_Iron', 'd_Refrigerator', 'd_Cooking_basics',
        'd_Bed_linens', 'd_Shampoo'
    ]
    
    # columns to keep
    columns = list(set([
        'accommodates', 'beds', 'review_scores_rating', 'host_is_superhost',
        'latitude', 'longitude', 'host_since', 'number_of_reviews',
        'availability_365', 'minimum_nights', 'maximum_nights', 'property_type',
        'room_type', 'beds', 'price'
    ] + top_amenities_columns))
    
    # Ensure the 'amenities' column is dropped
    columns = [col for col in columns if col != 'amenities']  # Just in case to ensure 'amenities' column is not included

    # Creating a new DataFrame with only the selected columns
    data = data[columns]
    
    # Proceed with cleaning as before
    data = data[data['price'] != 0]
    data = data.dropna(subset=['price'])
    data['price'] = data['price'].str.replace('$', '').str.replace(',', '').astype(float).astype(int)
    
    property_type_value_counts = data['property_type'].value_counts()
    to_remove = property_type_value_counts[property_type_value_counts < 100].index
    data = data[~data['property_type'].isin(to_remove)]

    property_dummies = pd.get_dummies(data['property_type'], prefix='d_type')
    data = pd.concat([data, property_dummies], axis=1)

    room_dummies = pd.get_dummies(data['room_type'], prefix='d_room')
    data = pd.concat([data, room_dummies], axis=1)

    data.rename(columns=lambda x: x.replace(" ", "_").lower(), inplace=True)
    
    data = data.rename(columns={
        'review_scores_rating': 'n_review_scores_rating',
        'host_since': 'date_host_start',
        'minimum_nights': 'n_minimum_nights',
        'accommodates': 'n_accommodates',
        'beds': 'n_beds',
        'availability_365': 'n_availability_365',
        'number_of_reviews': 'n_number_of_reviews',
        'maximum_nights': 'n_maximum_nights',
        'room_type': 'f_room_type',
        'property_type': 'f_property_type'
    })

    data['date_host_start'] = pd.to_datetime(data['date_host_start'])
    data['n_beds'] = data['n_beds'].fillna(0).astype(int)
    data['host_is_superhost'] = data['host_is_superhost'].map({'t': 1, 'f': 0}).fillna(0).astype(int)
    data['n_review_scores_rating'] = data['n_review_scores_rating'].fillna(0)
    d_columns = data.columns[data.columns.str.startswith('d_')]
    data[d_columns] = data[d_columns].astype(int)
    
    
    # resort the data by alphabetical order
    data = data.reindex(sorted(data.columns), axis=1)
    
    return data

# Assume 'data' is your DataFrame loaded earlier
data = airbnb_cleaner(data)


KeyError: 'amenities'

In [ ]:
data.dtypes

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10331 entries, 0 to 13133
Data columns (total 41 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   n_beds                              10331 non-null  int32         
 1   d_bed_linens                        10331 non-null  int32         
 2   d_smoke_alarm                       10331 non-null  int32         
 3   n_minimum_nights                    10331 non-null  int64         
 4   n_review_scores_rating              10331 non-null  float64       
 5   n_maximum_nights                    10331 non-null  int64         
 6   f_room_type                         10331 non-null  object        
 7   d_wifi                              10331 non-null  int32         
 8   d_hangers                           10331 non-null  int32         
 9   price                               10331 non-null  int32         
 10  n_number_of_reviews        

In [38]:
data.isna().sum().sum()

6

In [39]:
data.shape

(10331, 41)

### EDA

In [14]:
# copy a variable - purpose later, see at variable importance
data['n_accommodates_copy'] = data['n_accommodates']

***numerical variables***

In [15]:
# too long to display and read
data.describe().T

,count,mean,std,min,25%,50%,75%,max
id,11233.0,2.633452e+17,3.936411e+17,3.176000e+03,1.697287e+07,3.842267e+07,6.852340e+17,1.049080e+18
scrape_id,11233.0,2.023120e+13,0.000000e+00,2.023120e+13,2.023120e+13,2.023120e+13,2.023120e+13,2.023120e+13
description,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
host_id,11233.0,1.423954e+08,1.637434e+08,1.581000e+03,1.281804e+07,5.762884e+07,2.434834e+08,5.510794e+08
host_listings_count,11226.0,1.513264e+01,7.138824e+01,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,1.083000e+03
host_total_listings_count,11226.0,1.694539e+01,7.711085e+01,1.000000e+00,1.000000e+00,2.000000e+00,5.000000e+00,1.143000e+03
latitude,11233.0,5.251003e+01,3.284366e-02,5.236904e+01,5.249084e+01,5.250989e+01,5.253233e+01,5.265611e+01
longitude,11233.0,1.340320e+01,6.592717e-02,1.310758e+01,1.336761e+01,1.341224e+01,1.343815e+01,1.371796e+01
accommodates,11233.0,2.921303e+00,1.233958e+00,2.000000e+00,2.000000e+00,2.000000e+00,4.000000e+00,6.000000e+00
bathrooms,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***categorical variables***

In [18]:
data.room_type.value_counts()

room_type
Entire home/apt    7966
Private room       3078
Hotel room          114
Shared room          75
Name: count, dtype: int64

In [19]:
data.property_type.value_counts()

property_type
Entire rental unit                    6237
Private room in rental unit           2301
Entire condo                           722
Entire serviced apartment              358
Entire loft                            247
Room in hotel                          220
Private room in condo                  183
Private room in home                   127
Entire home                            126
Entire guesthouse                       67
Room in boutique hotel                  64
Shared room in hostel                   49
Private room in loft                    45
Entire vacation home                    45
Private room in hostel                  43
Room in aparthotel                      39
Room in serviced apartment              36
Private room in bed and breakfast       32
Private room in townhouse               29
Entire townhouse                        24
Houseboat                               23
Shared room in rental unit              20
Entire bungalow                         

In [20]:
data.number_of_reviews.value_counts()

number_of_reviews
0      2364
1       902
2       644
3       507
4       390
       ... 
493       1
653       1
655       1
435       1
387       1
Name: count, Length: 456, dtype: int64

In [21]:
data.neighbourhood_cleansed.value_counts()

neighbourhood_cleansed
Alexanderplatz               760
Frankfurter Allee Süd FK     612
Tempelhofer Vorstadt         554
Brunnenstr. Süd              487
Reuterstraße                 393
                            ... 
Allende-Viertel                3
MV 1                           3
Neu-Hohenschönhausen Süd       2
Hellersdorf-Süd                1
Neu-Hohenschönhausen Nord      1
Name: count, Length: 136, dtype: int64

***split train and test***
- train is where we do it all, incl CV

- first pick a smaller than usual training set so that models run faster and check if works
- if works, start anew without these two lines

In [22]:
data_train, data_holdout = train_test_split( data, train_size=0.7, random_state=42)

In [23]:
data_train.shape, data_holdout.shape

((7863, 76), (3370, 76))